In [5]:
# %%
## 1D General elliptic PDE of the following form: 
## -div( a(x) grad u(x)) + b(x) grad u(x) + c(x) u(x) = f(x) in [0,1] 
## a(x), b(x), c(x) are set to be constant functions 
## du_dn = g on the boundary 
## this version also contains using the tanh-activated shallow neural network to solve the PDE 
"""
log
Nov 17th 2024 Modified by Xiaofeng: 
added three functions   
1. select_discrete_dictionary
2. compute_l2_error 
3. compute_gradient_error

Nov 20th 2024 Modified by Xiaofeng 
1. use an efficient way to assemble the matrix that reuses previous matrices 
    - minimize_linear_layer_efficient

Todo: 
1. remove some redundant variable to save memory 
2. test a huge dictionary and huge quadrature points  
"""
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import sys
import os 
from scipy.sparse import linalg
from pathlib import Path
if torch.cuda.is_available():  
    device = "cuda" 
else:  
    device = "cpu" 

torch.set_default_dtype(torch.float64)
pi = torch.tensor(np.pi)
ZERO = torch.tensor([0.]).to(device)

###===============model parameters below================================
LAMBDA = -4 # c(x) = LAMBDA, if negative Helmholtz equation parameters
BETA = 5 ## convection term parameters 
DIMENSION = 3  ## dimension of the problem 
###===============model parameters above================================

## Define the neural network model
## already general in any dimension
class model_tanh(nn.Module):
    """ cosine shallow neural network
    Parameters: 
    input size: input dimension
    hidden_size1 : number of hidden layers 
    num_classes: output classes 
    """
    def __init__(self, input_size, hidden_size1, num_classes):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, num_classes,bias = False)
    def forward(self, x):
        u1 = self.fc2( torch.tanh(self.fc1(x)) )
        return u1
    
    def tanh_activation_dx(self,x): 
        return 1/torch.cosh(x)**2  
      
    def evaluate_derivative(self, x, i):
        u1 = self.fc2( self.tanh_activation_dx(self.fc1(x)) *self.fc1.weight.t()[i-1:i,:] )  
        return u1

class model(nn.Module):
    """ ReLU k shallow neural network
    Parameters: 
    input size: input dimension
    hidden_size1 : number of hidden layers 
    num_classes: output classes 
    k: degree of relu functions
    """
    def __init__(self, input_size, hidden_size1, num_classes,k = 1):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, num_classes,bias = False)
        self.k = k 
    def forward(self, x):
        u1 = self.fc2(F.relu(self.fc1(x))**self.k)
        return u1
    def evaluate_derivative(self, x, i):
        if self.k == 1:
            ## ZERO = torch.tensor([0.]).to(device)
            u1 = self.fc2(torch.heaviside(self.fc1(x),ZERO) * self.fc1.weight.t()[i-1:i,:] )
        else:
            u1 = self.fc2(self.k*F.relu(self.fc1(x))**(self.k-1) *self.fc1.weight.t()[i-1:i,:] )  
        return u1
    

def show_convergence_order_latex2(err_l2,err_h10,exponent,k=1,d=1): 
    neuron_nums = [2**j for j in range(2,exponent+1)]
    err_list = [err_l2[i] for i in neuron_nums ]
    err_list2 = [err_h10[i] for i in neuron_nums ] 
    l2_order = -1/2-(2*k + 1)/(2*d)
    h1_order =  -1/2-(2*(k-1)+ 1)/(2*d)
    print("neuron num  & \t $\|u-u_n \|_{{L^2}}$ & \t order $O(n^{{{:.2f}}})$  & \t $ | u -u_n |_{{H^1}}$ & \t order $O(n^{{{:.2f}}})$  \\\ \hline \hline ".format(l2_order,h1_order))
    for i, item in enumerate(err_list):
        if i == 0: 
            print("{} \t\t & {:.6f} &\t\t * & \t\t {:.6f} & \t\t *  \\\ \hline  \n".format(neuron_nums[i],item, err_list2[i] ) )   
        else: 
            print("{} \t\t &  {:.2e} &  \t\t {:.2f} &  \t\t {:.2e} &  \t\t {:.2f} \\\ \hline  \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2),err_list2[i] , np.log(err_list2[i-1]/err_list2[i])/np.log(2) ) )


## 3D ReLU$^3$ data 

In [6]:
dim = 3 
function_name = "cosine"
Nx = 50 
order = 2
exponent = 9
num_epochs = 2**exponent  
plot_freq = num_epochs 
rand_deter = 'rand'
activation = 'relu' 
relu_k = 3 


memory = 2**29
trial_num = 5 
for N_list in [[2**3,2**3,2**3]]:   
    errl2_trials = []
    err_h10_trials = [] 
    for trial in range(trial_num):
        print("trial: ", trial)
        N = np.prod(N_list)
        folder = './'
        filename = folder + 'errl2_OGA_3D_{}_{}_{}_neuron_{}_N_{}_rand_trial_{}.pt'.format(function_name,activation, relu_k, num_epochs,N,trial)
        errl2 = torch.load(filename) 
        filename = folder + 'err_h10_OGA_3D_{}_{}_{}_neuron_{}_N_{}_rand_trial_{}.pt'.format(function_name,activation, relu_k, num_epochs,N,trial)
        err_h10 = torch.load(filename)

        errl2_trials.append(errl2)
        err_h10_trials.append(err_h10)
    errl2_mean = torch.mean(torch.stack(errl2_trials), dim=0)
    err_h10_mean = torch.mean(torch.stack(err_h10_trials), dim=0)
    errl2_std = torch.std(torch.stack(errl2_trials), dim=0)
    err_h10_std = torch.std(torch.stack(err_h10_trials), dim=0)

    show_convergence_order_latex2(errl2_mean,err_h10_mean,exponent,relu_k,dim)

trial:  0
trial:  1
trial:  2
trial:  3
trial:  4
neuron num  & 	 $\|u-u_n \|_{L^2}$ & 	 order $O(n^{-1.67})$  & 	 $ | u -u_n |_{H^1}$ & 	 order $O(n^{-1.33})$  \\ \hline \hline 
4 		 & 0.684214 &		 * & 		 3.872194 & 		 *  \\ \hline  

8 		 &  4.36e-01 &  		 0.65 &  		 3.85e+00 &  		 0.01 \\ \hline  

16 		 &  9.21e-01 &  		 -1.08 &  		 3.89e+00 &  		 -0.01 \\ \hline  

32 		 &  4.91e+00 &  		 -2.42 &  		 4.56e+00 &  		 -0.23 \\ \hline  

64 		 &  1.17e+00 &  		 2.07 &  		 1.80e+00 &  		 1.34 \\ \hline  

128 		 &  8.35e-02 &  		 3.81 &  		 5.42e-01 &  		 1.73 \\ \hline  

256 		 &  1.15e-02 &  		 2.86 &  		 1.66e-01 &  		 1.71 \\ \hline  

512 		 &  1.75e-03 &  		 2.71 &  		 5.05e-02 &  		 1.71 \\ \hline  



## 3D tanh data 

In [7]:
dim = 3 
function_name = "cosine"
Nx = 50 
order = 2
exponent = 9
num_epochs = 2**exponent  
plot_freq = num_epochs 
rand_deter = 'rand'
memory = 2**29
activation = 'tanh' 
relu_k = 3 # not used if activation != relu 


trial_num = 5 
for N_list in [[2**3,2**3,2**3]]: # ,[2**6,2**6],[2**7,2**7] 
    errl2_trials = []
    err_h10_trials = []
    for trial in range(trial_num): 
        print("trial: ", trial)
        N = np.prod(N_list)
        folder = './'
        filename = folder + 'errl2_OGA_3D_{}_{}_neuron_{}_N_{}_rand_trial_{}.pt'.format(function_name,activation, num_epochs,N,trial)
        errl2 = torch.load(filename)
        filename = folder + 'err_h10_OGA_3D_{}_{}_neuron_{}_N_{}_rand_trial_{}.pt'.format(function_name,activation, num_epochs,N,trial)
        err_h10 = torch.load(filename)
        errl2_trials.append(errl2)
        err_h10_trials.append(err_h10)
    errl2_mean = torch.mean(torch.stack(errl2_trials), dim=0)
    err_h10_mean = torch.mean(torch.stack(err_h10_trials), dim=0)

    show_convergence_order_latex2(errl2_mean,err_h10_mean,exponent,relu_k,dim)


trial:  0
trial:  1
trial:  2
trial:  3
trial:  4
neuron num  & 	 $\|u-u_n \|_{L^2}$ & 	 order $O(n^{-1.67})$  & 	 $ | u -u_n |_{H^1}$ & 	 order $O(n^{-1.33})$  \\ \hline \hline 
4 		 & 0.954152 &		 * & 		 3.891471 & 		 *  \\ \hline  

8 		 &  2.82e+00 &  		 -1.56 &  		 4.57e+00 &  		 -0.23 \\ \hline  

16 		 &  8.93e+00 &  		 -1.66 &  		 7.14e+00 &  		 -0.64 \\ \hline  

32 		 &  4.81e+00 &  		 0.89 &  		 4.53e+00 &  		 0.66 \\ \hline  

64 		 &  1.01e+00 &  		 2.26 &  		 1.75e+00 &  		 1.37 \\ \hline  

128 		 &  1.10e-01 &  		 3.20 &  		 5.36e-01 &  		 1.70 \\ \hline  

256 		 &  6.23e-03 &  		 4.14 &  		 1.09e-01 &  		 2.29 \\ \hline  

512 		 &  4.16e-04 &  		 3.90 &  		 1.28e-02 &  		 3.10 \\ \hline  

